In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

In [34]:
deep_reload(src)

n = 1000
scale = (1e6, 1e6)
reference_distance = 200e3
seed = 129687083

graph = src.rng.random_graph(
    n,
    scale = scale,
    reference_distance = reference_distance,
    seed = seed,
)

origins = src.rng.random_nodes(graph, 1)
destinations = src.rng.random_nodes(graph, 10)
stations = src.rng.random_nodes(graph, 15)

In [35]:
deep_reload(src)

t0 = time.time()

path_costs, path_values, paths = src.routing.dijkstra(graph, origins)

print(time.time() - t0)

0.11629223823547363


In [36]:
deep_reload(src)

objective = src.routing.Vehicle()

t0 = time.time()

path_costs, path_values, paths = src.routing.dijkstra(
    graph, origins, objective = objective)

print(time.time() - t0)

0.5952949523925781


In [37]:
len(path_costs)

428

In [40]:
t0 = time.time()

r = nx.single_source_dijkstra(graph, origins[0], weight = 'time')
print(time.time() - t0)

0.09750843048095703


In [41]:
t0 = time.time()

r = nx.single_source_bellman_ford(graph, origins[0], weight = 'time')
print(time.time() - t0)

0.10377264022827148


In [187]:
graph = src.graph.graph_from_json('Outputs/graph.json')

In [303]:
for source, adj in graph._adj.items():
    for target, edge in adj.items():
        
        graph._node[source]['time'] = 0
        graph._node[source]['distance'] = 0
        graph._node[source]['price'] = 0

        graph._node[target]['time'] = 0
        graph._node[target]['distance'] = 0
        graph._node[target]['price'] = 0

        edge['price'] = 0
        if ('station' in source) and ('station' in target):
            
            edge['type'] = 'inter_station'
            
        else:
            
            edge['type'] = 'destination'

In [189]:
graph.number_of_nodes()

1921

In [240]:
deep_reload(src)

objective = src.dijkstra.Objective(field = 'time')

t0 = time.time()

r = src.dijkstra.dijkstra(graph, ['Fresno'], objective = objective)

print(time.time() - t0)

1.33335542678833


In [241]:
{k: v / 3600 for k, v in r[0].items() if 'station' not in k}

{'Fresno': 0.0,
 'Bakersfield': 1.608267669830169,
 'Stockton': 2.0049909396159387,
 'San Jose': 2.5324649794649785,
 'Sacramento': 2.7117650058275027,
 'San Francisco': 2.9574284049284016,
 'Los Angeles': 3.2643437812187788,
 'Chico': 4.307111145798643,
 'I-80 to Reno (State Line)': 4.586502074314568,
 'Redding': 5.141404012654007,
 'San Diego': 5.253800990675983,
 'I-15 to Las Vegas (State Line)': 5.820441589660336,
 'Yreka': 6.627662493756237,
 'I-10 to Phoenix (State Line)': 6.672202103452094,
 'Crescent City': 8.611122946497934}

In [298]:
deep_reload(src)

objective = src.routing.Vehicle(capacity = 80 * 3.6e6)

t0 = time.time()

r = src.dijkstra.dijkstra(graph, ['Fresno'], objective = objective, return_paths = True)

print(time.time() - t0)

6.382099866867065


In [299]:
{k: v / 3600 for k, v in r[0].items() if 'station' not in k}

{'Fresno': 0.0,
 'Bakersfield': 1.608267669830169,
 'Stockton': 2.0049909396159387,
 'San Jose': 2.5324649794649785,
 'Sacramento': 2.7117650058275027,
 'San Francisco': 2.9574284049284016,
 'Los Angeles': 3.2643437812187788,
 'Chico': 4.307111145798643,
 'I-80 to Reno (State Line)': 4.586502074314568}

In [306]:
deep_reload(src)

objective = src.routing.Vehicle(capacity = 80 * 3.6e6)

t0 = time.time()

r = src.bellman.bellman(graph, ['Fresno'], objective = objective, return_paths = True)

print(time.time() - t0)

0.6208980083465576


In [307]:
{k: v / 3600 for k, v in r[0].items() if 'station' not in k}

{'Fresno': 0.0,
 'Sacramento': 2.7789994380619363,
 'Stockton': 2.0456975940725926,
 'San Francisco': 3.070305957930957,
 'San Jose': 2.606983488733492,
 'Bakersfield': 1.6082676698301692,
 'Los Angeles': 3.292798326673327,
 'Chico': 4.387682824120326,
 'I-80 to Reno (State Line)': 4.6272087287712305}

In [310]:
objective.range / 1.609e3

325.4421153737499

In [89]:
t0 = time.time()

r = nx.single_source_dijkstra(g, 'Fresno', weight = 'time')
print(time.time() - t0)

0.6918160915374756


In [90]:
{k: v / 3600 for k, v in r[0].items() if 'station' not in k}

{'Fresno': 0.0,
 'Bakersfield': 1.608267669830169,
 'Stockton': 2.0049909396159387,
 'San Jose': 2.5324649794649785,
 'Sacramento': 2.7117650058275027,
 'San Francisco': 2.9574284049284016,
 'Los Angeles': 3.2643437812187788,
 'Chico': 4.307111145798643,
 'I-80 to Reno (State Line)': 4.586502074314568,
 'Redding': 5.141404012654007,
 'San Diego': 5.253800990675983,
 'I-15 to Las Vegas (State Line)': 5.820441589660336,
 'Yreka': 6.627662493756237,
 'I-10 to Phoenix (State Line)': 6.672202103452094,
 'Crescent City': 8.611122946497934}

In [91]:
t0 = time.time()

r = nx.single_source_bellman_ford(g, 'Fresno', weight = 'time')
print(time.time() - t0)

0.6678893566131592


In [92]:
{k: v / 3600 for k, v in r[0].items() if 'station' not in k}

{'Fresno': 0.0,
 'Sacramento': 2.7117650058275027,
 'Stockton': 2.0049909396159387,
 'San Francisco': 2.9574284049284016,
 'San Jose': 2.5324649794649785,
 'Bakersfield': 1.608267669830169,
 'Los Angeles': 3.2643437812187788,
 'Redding': 5.141404012654007,
 'Chico': 4.307111145798643,
 'I-80 to Reno (State Line)': 4.586502074314568,
 'I-15 to Las Vegas (State Line)': 5.820441589660336,
 'San Diego': 5.253800990675983,
 'Yreka': 6.627662493756237,
 'I-10 to Phoenix (State Line)': 6.672202103452094,
 'Crescent City': 8.611122946497934}